In [2]:
import pandas as pd
import numpy as np

from shapely.geometry import Point, shape, mapping
from shapely.ops import unary_union

import matplotlib.pyplot as plt
import seaborn as sns

#from geoband.API import *
import folium
import json
import geopandas as gpd
from geopandas import gpd

from folium import plugins
import re
from pandas import ExcelFile
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
pd.options.display.max_rows = 100
pd.options.display.max_columns = 40
import numpy as np
import os,random, math
from tqdm import tqdm
from copy import deepcopy
from collections import Counter

# Visualization
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc
plt.rcParams['font.family'] = 'NanumGothic'
import platform
if platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
    rc('font', family='AppleGothic')

matplotlib.rcParams['axes.unicode_minus'] = False

# from dataprep.eda import plot, plot_correlation, plot_missing

import plotly 
import plotly.express as px
# from plotly import tools, subplots
# import plotly.offline as py
# py.init_notebook_mode(connected=True)
# from plotly.offline import init_notebook_mode, iplot
# import plotly.graph_objs as go
# import plotly.express as px

# Warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')


In [3]:
test = pd.read_csv("/Users/seungji/Desktop/Dacon/Daesamanlap/ProcessedData/merged_test.csv")
train = pd.read_csv("/Users/seungji/Desktop/Dacon/Daesamanlap/ProcessedData/merged_train.csv")

In [4]:
tr_gr = train.groupby("단지코드").mean().reset_index()
tst_gr = test.groupby("단지코드").mean().reset_index()
tr_gr["전용면적_std"] = train.groupby("단지코드").std().reset_index()["전용면적"]
tr_gr["전용면적별세대수_std"] = train.groupby("단지코드").std().reset_index()["전용면적별세대수"]
tr_gr["임대보증금_std"] = train.groupby("단지코드").std().reset_index()["임대보증금"]
tr_gr["임대료_std"] = train.groupby("단지코드").std().reset_index()["임대료"]
tst_gr["전용면적_std"] = test.groupby("단지코드").std().reset_index()["전용면적"]
tst_gr["전용면적별세대수_std"] = test.groupby("단지코드").std().reset_index()["전용면적별세대수"]
tst_gr["임대보증금_std"] = test.groupby("단지코드").std().reset_index()["임대보증금"]
tst_gr["임대료_std"] = test.groupby("단지코드").std().reset_index()["임대료"]
tr_re = pd.concat([train.단지코드,pd.get_dummies(train["지역"])],axis=1).groupby("단지코드").sum()
tst_re = pd.concat([test.단지코드,pd.get_dummies(test["지역"])],axis=1).groupby("단지코드").sum()
tr_re[tr_re>0] = 1
tst_re[tst_re>0] = 1
tr_gr = tr_gr.merge(tr_re, left_on = "단지코드",right_on ="단지코드",how = "left")
tst_gr =tst_gr.merge(tst_re, left_on = "단지코드",right_on ="단지코드",how = "left")
tr_apart = pd.concat([train.단지코드,pd.get_dummies(train["임대건물구분"])],axis=1).groupby("단지코드").sum().reset_index()
tst_apart = pd.concat([test.단지코드,pd.get_dummies(test["임대건물구분"])],axis=1).groupby("단지코드").sum().reset_index()
tr_gr = tr_gr.merge(tr_apart, left_on = "단지코드",right_on ="단지코드",how = "left")
tst_gr =tst_gr.merge(tst_apart, left_on = "단지코드",right_on ="단지코드",how = "left")
tr_gong = pd.concat([train.단지코드, pd.get_dummies(train["공급유형"])],axis =1)
tst_gong = pd.concat([test.단지코드, pd.get_dummies(test["공급유형"])],axis =1)
tr_gong["공공임대(5년,10년)"] = tr_gong["공공임대(5년)"]+tr_gong["공공임대(10년)"]
tst_gong["공공임대(5년,10년)"] = tr_gong["공공임대(10년)"]
tr_gong = tr_gong[['단지코드',   '공공임대(50년)',  '공공임대(분납)',
       '국민임대', '영구임대', '임대상가',  '행복주택', '공공임대(5년,10년)']]
tst_gong = tst_gong[['단지코드',   '공공임대(50년)',  '공공임대(분납)',
       '국민임대', '영구임대', '임대상가',  '행복주택', '공공임대(5년,10년)']]
tr_gong = tr_gong.groupby("단지코드").sum()
tst_gong = tst_gong.groupby("단지코드").sum()
tr_gong[tr_gong>0] = 1
tst_gong[tst_gong>0] = 1
tr_gong = tr_gong.reset_index()
tst_gong = tst_gong.reset_index()

tr_gr = tr_gr.merge(tr_gong, left_on = "단지코드",right_on ="단지코드",how = "left")
tst_gr =tst_gr.merge(tst_gong, left_on = "단지코드",right_on ="단지코드",how = "left")

tr_ja = pd.concat([train.단지코드,pd.get_dummies(train["자격유형"])],axis=1).groupby("단지코드").sum()
tst_ja = pd.concat([test.단지코드,pd.get_dummies(test["자격유형"])],axis=1).groupby("단지코드").sum()

tr_ja[tr_ja>0] = 1
tst_ja[tst_ja>0] = 1

tr_ja = tr_ja[tst_ja.columns]
tr_ja = tr_ja.reset_index()
tst_ja = tst_ja.reset_index()

tr_gr = tr_gr.merge(tr_ja, left_on = "단지코드",right_on ="단지코드",how = "left")
tst_gr =tst_gr.merge(tst_ja, left_on = "단지코드",right_on ="단지코드",how = "left")
tr_gr =tr_gr.drop("서울특별시",axis=1)

tr_gr.loc[tr_gr.전용면적_std.isna(),"전용면적_std"] = 0
tr_gr.loc[tr_gr.전용면적별세대수_std.isna(),"전용면적별세대수_std"] = 0
tr_gr.loc[tr_gr.임대보증금_std.isna(),"임대보증금_std"] = 0
tr_gr.loc[tr_gr.임대료_std.isna(),"임대료_std"] = 0



In [5]:
tr_gr.to_csv("tr_1dim.csv",index=False)
tst_gr.to_csv("tst_1dim.csv",index=False)


In [106]:

X_tr = tr_gr.drop(columns = ["단지코드","등록차량수"])

y = tr_gr["등록차량수"]
X_tst = tst_gr.drop(columns =["단지코드"])
X_tr = X_tr[X_tst.columns]


In [107]:
X_tr_g = gpd.GeoDataFrame(X_tr,
    crs={'init': 'epsg:4326'},
    geometry=[Point(xy) for xy in zip(X_tr.경도, X_tr.위도)])
X_tst_g = gpd.GeoDataFrame(X_tst,
    crs={'init': 'epsg:4326'},
    geometry=[Point(xy) for xy in zip(X_tst.경도, X_tst.위도)])

In [108]:
coords_tr = np.array(list(zip(X_tr['위도'], X_tr['경도'])))
coords_tst = np.array(list(zip(X_tst['위도'], X_tst['경도'])))

In [109]:
X_tr = X_tr.drop(columns = ["위도","경도",'C', 'D',"임대료_std","임대상가"])
X_tst = X_tst.drop(columns = ["위도","경도",'C', 'D',"임대료_std","임대상가"])
X_tr = X_tr.drop(columns = ["geometry"])
X_tst = X_tst.drop(columns = ["geometry"])

In [100]:
X_tr.columns

Index(['총세대수', '전용면적', '전용면적별세대수', '공가수', '지하철역', '버스정류장', '단지내주차면수', '임대보증금',
       '임대료', '연면적', 'subway_dist', '환승역 수', '총인구수', '세대당_인구', '남/여비율',
       '남/여_0~19세', '남/여_20~39세', '남/여_40~69세', '남/여_70세이상', '0~19세_비율',
       '20~39세_비율', '40~69세_비율', '70세이상_비율', '전용면적_std', '전용면적별세대수_std',
       '임대보증금_std', '강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시',
       '부산광역시', '세종특별자치시', '울산광역시', '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도',
       '상가', '아파트', '공공임대(50년)', '공공임대(분납)', '국민임대', '영구임대', '임대상가', '행복주택',
       '공공임대(5년,10년)', 'A', 'E', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
      dtype='object')

y = datas["finedust"]
X = datas.iloc[:,range(3,24)] 

In [101]:
for (idx1, i) in enumerate(X_tr.corr().columns):
    for (idx2, j) in enumerate(X_tr.corr().columns):
        if (X_tr.corr().iloc[idx1,idx2]>=0.7)&(X_tr.corr().iloc[idx1,idx2]<1.0):
            print(i,"+",j,": ",X_tr.corr().iloc[idx1,idx2])

총세대수 + 단지내주차면수 :  0.7591780182644089
전용면적 + 공공임대(5년,10년) :  0.7058024748880461
단지내주차면수 + 총세대수 :  0.7591780182644089
subway_dist + 제주특별자치도 :  0.7636006598844339
세대당_인구 + 0~19세_비율 :  0.8430115273629925
남/여비율 + 남/여_40~69세 :  0.8330993369676842
남/여_40~69세 + 남/여비율 :  0.8330993369676842
0~19세_비율 + 세대당_인구 :  0.8430115273629925
전용면적_std + 상가 :  0.7965289083608125
전용면적_std + 임대상가 :  0.7083976185674922
제주특별자치도 + subway_dist :  0.7636006598844339
상가 + 전용면적_std :  0.7965289083608125
상가 + 임대상가 :  0.8425713066069851
영구임대 + 임대상가 :  0.7441182943561417
임대상가 + 전용면적_std :  0.7083976185674922
임대상가 + 상가 :  0.8425713066069851
임대상가 + 영구임대 :  0.7441182943561417
행복주택 + J :  0.7539812971182063
공공임대(5년,10년) + 전용면적 :  0.7058024748880461
J + 행복주택 :  0.7539812971182063


In [102]:
X_tr.corr()

,총세대수,전용면적,전용면적별세대수,공가수,지하철역,버스정류장,단지내주차면수,임대보증금,임대료,연면적,subway_dist,환승역 수,총인구수,세대당_인구,남/여비율,남/여_0~19세,남/여_20~39세,남/여_40~69세,남/여_70세이상,0~19세_비율,...,충청북도,상가,아파트,공공임대(50년),공공임대(분납),국민임대,영구임대,임대상가,행복주택,"공공임대(5년,10년)",A,E,G,H,I,J,K,L,M,N
총세대수,1.0000,0.0049,0.3791,0.2970,0.1145,0.1031,0.7592,0.0720,0.2496,0.1174,-0.2800,-0.0406,0.2660,0.2394,-0.1350,0.2130,-0.2444,-0.2307,0.1145,0.2189,...,-0.0567,0.3209,0.3716,-0.0204,0.0261,-0.0782,0.2697,0.2094,-0.0705,0.0186,-0.1928,0.0663,0.0482,0.2370,0.1729,-0.0747,-0.0225,-0.1085,0.0218,0.0972
전용면적,0.0049,1.0000,0.0844,-0.1946,-0.0118,0.0350,0.1729,0.4174,0.1348,0.1528,-0.1431,-0.0570,0.0797,0.1341,-0.0077,0.1354,-0.0621,-0.0160,0.0854,0.1436,...,0.0261,-0.0443,-0.0942,-0.0482,0.1723,-0.1429,-0.2503,-0.0859,-0.4288,0.7058,0.4462,-0.0567,-0.0139,-0.2193,-0.2411,-0.3229,-0.1718,-0.1925,-0.0657,-0.1213
전용면적별세대수,0.3791,0.0844,1.0000,0.2725,-0.0084,0.0777,0.4568,-0.0021,-0.0898,0.0482,-0.1577,-0.0461,0.1377,0.1743,-0.0371,0.1944,-0.1615,-0.1069,0.1204,0.2106,...,-0.0256,-0.2149,-0.2649,0.2482,-0.0407,0.0985,-0.1844,-0.2379,-0.1321,0.0451,0.1581,0.0389,-0.0342,0.1154,0.0036,-0.0788,-0.0540,-0.1132,-0.0315,-0.0235
공가수,0.2970,-0.1946,0.2725,1.0000,-0.1191,0.0621,0.3844,0.0850,-0.1325,0.0593,-0.0863,-0.0936,0.2085,0.1094,0.1588,0.3028,-0.0483,0.1099,0.0688,0.2502,...,0.1323,-0.1415,0.1469,-0.1568,-0.0951,0.0614,-0.1931,-0.1995,0.2571,-0.0704,-0.1284,0.1969,0.0380,0.1181,-0.0522,0.2050,0.2236,-0.0184,0.0890,0.1047
지하철역,0.1145,-0.0118,-0.0084,-0.1191,1.0000,0.1098,0.0133,0.0423,-0.0144,0.1260,-0.2449,0.0253,0.0289,-0.0477,-0.1087,0.0417,-0.0966,-0.1217,0.0822,-0.1690,...,-0.0993,0.1550,0.0491,0.0536,0.0406,-0.0697,0.1236,0.1178,-0.0267,-0.0269,-0.0486,-0.0088,0.0621,-0.0352,0.0660,-0.0432,-0.0393,0.0577,-0.0175,-0.0304
버스정류장,0.1031,0.0350,0.0777,0.0621,0.1098,1.0000,0.1007,0.1546,0.0199,-0.0254,-0.0631,-0.0683,0.1851,0.1800,-0.0040,-0.0076,-0.0915,-0.1017,-0.0162,0.1553,...,-0.0829,-0.0055,0.0106,0.0918,0.0090,-0.0199,0.0215,0.0593,-0.0382,0.0099,0.0138,0.0876,-0.0297,0.0208,-0.0342,-0.0085,-0.0589,-0.0231,-0.0297,-0.0213
단지내주차면수,0.7592,0.1729,0.4568,0.3844,0.0133,0.1007,1.0000,0.3203,-0.0867,0.2496,-0.2640,-0.0116,0.3882,0.4289,-0.0945,0.3738,-0.3767,-0.1947,0.2459,0.4551,...,0.0077,-0.2175,0.4987,-0.1323,0.1516,0.1500,-0.1718,-0.2991,-0.1261,0.2433,0.1456,0.0635,0.0585,0.2394,0.1134,-0.1152,-0.0469,-0.1148,0.0066,0.0599
임대보증금,0.0720,0.4174,-0.0021,0.0850,0.0423,0.1546,0.3203,1.0000,-0.1215,0.2560,-0.1915,0.0615,0.2899,0.2903,-0.0971,0.2069,-0.2826,-0.1347,0.3314,0.2012,...,-0.0422,-0.2051,0.2872,-0.1144,0.4133,-0.2471,-0.2209,-0.2402,0.1299,0.5384,0.0791,0.2328,0.0812,0.0035,-0.0194,0.0514,0.1468,0.0500,0.0065,0.0257
임대료,0.2496,0.1348,-0.0898,-0.1325,-0.0144,0.0199,-0.0867,-0.1215,1.0000,-0.0412,-0.0148,-0.0439,-0.0673,-0.1306,-0.0331,-0.0904,0.0595,-0.0541,-0.0440,-0.1440,...,-0.0445,0.4995,-0.1118,0.0615,0.0022,-0.2847,0.3464,0.4760,-0.0564,0.0408,-0.2213,-0.0405,-0.0037,-0.0411,-0.0408,-0.0404,-0.0299,-0.0240,-0.0089,-0.0161
연면적,0.1174,0.1528,0.0482,0.0593,0.1260,-0.0254,0.2496,0.2560,-0.0412,1.0000,-0.1348,0.0613,0.0702,0.0808,-0.0765,0.1139,-0.1724,-0.0328,0.1175,0.0992,...,0.0100,-0.0889,0.1695,-0.0193,0.2305,-0.0589,-0.0690,-0.1119,-0.0646,0.2391,0.1508,0.0231,0.0281,-0.0779,0.0301,-0.0057,-0.0635,-0.0760,-0.0353,-0.0331


In [110]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_tr)

X_tr = scaler.transform(X_tr)
X_tst = scaler.transform(X_tst)

In [111]:
y = np.array(y).reshape((-1,1))

y_tst = np.array(y_tst).reshape((-1,1))

In [55]:
import libpysal as ps
from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
from mgwr.utils import compare_surfaces, truncate_colorm

ImportError: cannot import name 'truncate_colorm' from 'mgwr.utils' (/Users/seungji/opt/anaconda3/lib/python3.8/site-packages/mgwr/utils.py)

In [113]:
bw = Sel_BW(coords_tr, y, X_tr, fixed = False, kernel = "Gaussian").search(criterion = 'AIC')
print(bw)

183.0


In [116]:
gwr_model = GWR(coords_tr,y,X_tr,bw = 183.0)
gwr_results = gwr_model.fit()
print(gwr_results.resid_ss)

LinAlgError: Matrix is singular.

In [ ]:
gwr_results.summary()

In [ ]:
y_tr_pred = gwr_model.predict(points = coords_tr, P = X_tr)
y_tr_pred = y_tr_pred.predictions
mean_squared_error(y_tr,y_tr_pred)

In [ ]:
y_tst_pred = gwr_model.predict(points = coords_tst, P = X_tst)

In [ ]:
y_tst_pred = y_tst_pred.predictions